In [4]:
from pyspark.sql import  SparkSession

In [5]:
spark = SparkSession.builder.appName("prod").getOrCreate()
spark

In [7]:
df1 = spark.range(2,10000000,2)
df2 = spark.range(2,10000000,4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

In [8]:
step4.collect()

[Row(sum(id)=2500000000000)]

In [9]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#19L)])
+- Exchange SinglePartition, true, [id=#66]
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#19L)])
      +- *(6) Project [id#19L]
         +- *(6) SortMergeJoin [id#19L], [id#13L], Inner
            :- *(3) Sort [id#19L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#19L, 200), true, [id=#50]
            :     +- *(2) Project [(id#11L * 5) AS id#19L]
            :        +- Exchange RoundRobinPartitioning(5), false, [id=#46]
            :           +- *(1) Range (2, 10000000, step=2, splits=8)
            +- *(5) Sort [id#13L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#13L, 200), true, [id=#57]
                  +- Exchange RoundRobinPartitioning(6), false, [id=#56]
                     +- *(4) Range (2, 10000000, step=4, splits=8)




In [3]:
spark.stop()